In [91]:
import warnings
warnings.filterwarnings('ignore')

In [92]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from scikeras.wrappers import KerasClassifier
from keras.constraints import MaxNorm

In [93]:
# Epoch y Batch Optimization
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(64, input_shape=(5,), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))


    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)
# load dataset
dataset = pd.read_excel("default_cc_clients.xlsx", header=1)
# split into input (X) and output (Y) variables
dataset.drop(['ID', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], axis=1, inplace=True)
dataset.drop(['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6'], axis=1, inplace=True)
dataset.drop(['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], axis=1, inplace=True)

In [94]:
dataset.head()
X = dataset.iloc[:,0:5]
Y = dataset.iloc[:,5]

In [95]:
X

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE
0,20000,2,2,1,24
1,120000,2,2,2,26
2,90000,2,2,2,34
3,50000,2,2,1,37
4,50000,1,2,1,57
...,...,...,...,...,...
29995,220000,1,3,1,39
29996,150000,1,3,2,43
29997,30000,1,2,2,37
29998,80000,1,3,1,41


In [96]:
Y

0        1
1        1
2        0
3        0
4        0
        ..
29995    0
29996    0
29997    1
29998    1
29999    1
Name: default payment next month, Length: 30000, dtype: int64

In [97]:
from sklearn.model_selection import train_test_split
X = np.asarray(X)
Y = np.asarray(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

In [98]:

# create model
model = KerasClassifier(build_fn=create_model, epochs=70, batch_size=32, verbose=2)

# define the grid search parameters
batch_size = [20, 40, 60]
epochs = [10, 40, 60]
param_grid = dict(batch_size=batch_size, epochs=epochs)

In [99]:
# Grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
1500/1500 - 3s - loss: 0.5277 - accuracy: 0.7716 - 3s/epoch - 2ms/step
Epoch 2/10
1500/1500 - 2s - loss: 0.5159 - accuracy: 0.7786 - 2s/epoch - 1ms/step
Epoch 3/10
1500/1500 - 2s - loss: 0.5156 - accuracy: 0.7788 - 2s/epoch - 1ms/step
Epoch 4/10
1500/1500 - 2s - loss: 0.5160 - accuracy: 0.7789 - 2s/epoch - 2ms/step
Epoch 5/10
1500/1500 - 2s - loss: 0.5158 - accuracy: 0.7788 - 2s/epoch - 2ms/step
Epoch 6/10
1500/1500 - 2s - loss: 0.5161 - accuracy: 0.7785 - 2s/epoch - 1ms/step
Epoch 7/10
1500/1500 - 2s - loss: 0.5158 - accuracy: 0.7788 - 2s/epoch - 2ms/step
Epoch 8/10
1500/1500 - 2s - loss: 0.5160 - accuracy: 0.7788 - 2s/epoch - 1ms/step
Epoch 9/10
1500/1500 - 2s - loss: 0.5154 - accuracy: 0.7788 - 2s/epoch - 1ms/step
Epoch 10/10
1500/1500 - 2s - loss: 0.5157 - accuracy: 0.7787 - 2s/epoch - 1ms/step
Best: 0.778800 using {'batch_size': 20, 'epochs': 10}
0.778800 (0.000000) with: {'batch_size': 20, 'epochs': 10}
0.778800 (0.000000) with: {'batch_size': 20, 'epochs': 40}
0.22120

In [101]:
# Random search
grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/10
1500/1500 - 3s - loss: 0.5284 - accuracy: 0.7692 - 3s/epoch - 2ms/step
Epoch 2/10
1500/1500 - 3s - loss: 0.5164 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 3/10
1500/1500 - 3s - loss: 0.5163 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 4/10
1500/1500 - 3s - loss: 0.5165 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 5/10
1500/1500 - 3s - loss: 0.5167 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 6/10
1500/1500 - 3s - loss: 0.5165 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 7/10
1500/1500 - 3s - loss: 0.5156 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 8/10
1500/1500 - 3s - loss: 0.5163 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 9/10
1500/1500 - 3s - loss: 0.5154 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Epoch 10/10
1500/1500 - 3s - loss: 0.5155 - accuracy: 0.7788 - 3s/epoch - 2ms/step
Best: 0.778800 using {'epochs': 10, 'batch_size': 20}
0.778800 (0.000000) with: {'epochs': 10, 'batch_size': 20}
0.407067 (0.262855) with: {'epochs': 40, 'batch_size': 20}
0.59293